## Schedule Scrapper

### Basic Libraries Setup

In [156]:
# Libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [157]:
# page url
url = 'https://www.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches'

In [158]:
# User Agent is for Genuinue user information
HEADERS = ({'User-Agent' :'https://explore.whatismybrowser.com/useragents/parse/?analyse-my-user-agent=yes','Accept-Language':'en-US en;q=0.5'})

In [159]:
webpage = requests.get(url,headers = HEADERS)
webpage # Response should be 200


<Response [200]>

### Main Soup

In [160]:
print(f'Webpage Content : \n{webpage.content[:400]} \n\n Type : {type(webpage.content)}')

Webpage Content : 
b'\r\n\r\n<!DOCTYPE html><html lang="en" itemscope itemtype="http://schema.org/WebPage"><head><meta charset="utf-8"><script>var is_mobile = /symbian|tizen|midp|uc(web|browser)|MSIE (5.0|6.0|7.0|8.0)|tablet/i.test(navigator.userAgent);\tif(is_mobile && window.location.hostname != "www1.cricbuzz.com") window.location.hostname = "m.cricbuzz.com";</script><style>html{scroll-behavior: smooth;}\tbody{background' 

 Type : <class 'bytes'>


In [161]:
main_soup = BeautifulSoup(webpage.content,'html.parser') # to html
main_soup.text[:400] # First 00 characters in soup


'\n Indian Premier League 2024 schedule, live scores and results | Cricbuzz.com  ✖Live ScoresScheduleArchivesNewsAll Stories  Premium Editorials Latest NewsTopicsSpotlightOpinionsSpecialsStats & AnalysisInterviewsLive BlogsHarsha BhogleSeries  ICC Mens T20 World Cup 2024 Indian Premier League 2024 Major League Cricket 2024 T20 Blast 2024 The Hundred Mens Competition 2024 India tour of Zimbabwe, 2024'

In [162]:
match_links = main_soup.find_all('a',attrs={'class':'text-hvr-underline'})
venue_links = main_soup.find_all('div',attrs={'class':'text-gray'})
winner = main_soup.find_all('a',attrs={'class':'cb-text-complete'})

#CrossCheck 1st check First Match details
print(f'Match Details \n : {match_links[:3]}\n')
print(f'Venue Details \n : {venue_links[:3]}\n')
print(f'Winner  Details \n : {winner[:3]}\n')

Match Details 
 : [<a class="text-hvr-underline" href="/cricket-scores/89654/csk-vs-rcb-1st-match-indian-premier-league-2024" title="CHENNAI SUPER KINGS vs ROYAL CHALLENGERS BENGALURU Live Cricket Score and ball by ball commentary"><span>CHENNAI SUPER KINGS vs ROYAL CHALLENGERS BENGALURU, 1st Match</span></a>, <a class="text-hvr-underline" href="/cricket-scores/89661/pbks-vs-dc-2nd-match-indian-premier-league-2024" title="PUNJAB KINGS vs DELHI CAPITALS Live Cricket Score and ball by ball commentary"><span>PUNJAB KINGS vs DELHI CAPITALS, 2nd Match</span></a>, <a class="text-hvr-underline" href="/cricket-scores/89665/kkr-vs-srh-3rd-match-indian-premier-league-2024" title="KOLKATA KNIGHT RIDERS vs SUNRISERS HYDERABAD Live Cricket Score and ball by ball commentary"><span>KOLKATA KNIGHT RIDERS vs SUNRISERS HYDERABAD, 3rd Match</span></a>]

Venue Details 
 : [<div class="text-gray"> <span> 74 T20s </span> <span class="cb-nav-dt"> . </span> <span>Mar 22</span> <span> - May 26</span> </div>, <

In [163]:
# 74 Match links
#match_links[:73] # Q2 is gng on so 74-1

In [164]:
print(f'match_links : {len(match_links)}') # need to remove last url 74 only
print(f'venue_links : {len(venue_links)}') # Venues will be double (Stadium, Venue)

match_links : 76
venue_links : 149


### Match Schedule Extraction

In [165]:
match_links[0] # 1st Match Details

<a class="text-hvr-underline" href="/cricket-scores/89654/csk-vs-rcb-1st-match-indian-premier-league-2024" title="CHENNAI SUPER KINGS vs ROYAL CHALLENGERS BENGALURU Live Cricket Score and ball by ball commentary"><span>CHENNAI SUPER KINGS vs ROYAL CHALLENGERS BENGALURU, 1st Match</span></a>

In [166]:
match_links[0].text

'CHENNAI SUPER KINGS vs ROYAL CHALLENGERS BENGALURU, 1st Match'

In [167]:
start = match_links[0].text.index('vs') # First Team Name will come before vs

In [168]:
end = match_links[0].text.index(',') # Second Team Name will come before comma

In [169]:
match_links[0].text[:start].strip(),match_links[0].text[start+2:end].strip()

('CHENNAI SUPER KINGS', 'ROYAL CHALLENGERS BENGALURU')

In [170]:
team_1,team_2 = match_links[0].text[:start].strip(),match_links[0].text[start+2:end].strip()

In [171]:
"".join([word[0] for word in team_1.split(' ')]),"".join([word[0] for word in team_2.split(' ')]) # Intials

('CSK', 'RCB')

In [172]:
match_List = []
team_1_List = []
team_2_List = []
def match_schedule(match_links):
    '''
    This Function will Extract Match Details from Each Matchlinks
    Such as Match Fixture and Team Names

    Input : Match_links in HTML Format
    Output : Match_Schedule in separate list

    Sample Input : 'ROYAL CHALLENGERS BENGALURU VS CHENNAI SUPER KINGS, 1st Match'
    '''
    # For Easy Understanding please see above Input
    for i in match_links[:74]: # Currently coding
        row = i.text # Extracting only text from HTML

        first = row.index('vs') # First Team Name will come before v in "VS"
        end = row.find(',')  # This is placeholder for each match fixture

        match_ = row[:end].upper()
        team_1 = row[:first].strip() # Removing spaces & Capitalizing
        team_2 = row[first+2:end].strip()
        team_1,team_2 = "".join([word[0] for word in team_1.split(' ')]),"".join([word[0] for word in team_2.split(' ')]) # Only Intials of Each Team "MUMBAI INDIANS"--> "MI"

        match_List.append(match_)
        team_1_List.append(team_1)
        team_2_List.append(team_2)

    # Cross Check
    print(f'Match Details : {match_List[:3]} {len(match_List)}')
    print(f'Team_1 Names : {team_1_List[:3]} {len(team_1_List)}')
    print(f'Team_2 Names : {team_2_List[:3]}{len(team_2_List)}')

match_schedule(match_links)

Match Details : ['CHENNAI SUPER KINGS VS ROYAL CHALLENGERS BENGALURU', 'PUNJAB KINGS VS DELHI CAPITALS', 'KOLKATA KNIGHT RIDERS VS SUNRISERS HYDERABAD'] 74
Team_1 Names : ['CSK', 'PK', 'KKR'] 74
Team_2 Names : ['RCB', 'DC', 'SH']74


### Venue & City Extraction

In [173]:
venue_links[:4],len(venue_links)

([<div class="text-gray"> <span> 74 T20s </span> <span class="cb-nav-dt"> . </span> <span>Mar 22</span> <span> - May 26</span> </div>,
  <div class="text-gray">MA Chidambaram Stadium, Chennai</div>,
  <div class="cb-font-12 text-gray"><span>02:30 PM </span>GMT	/<span> 08:00 PM</span> LOCAL</div>,
  <div class="text-gray">Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh</div>],
 149)

In [174]:
venue_links[3].text.strip()

'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh'

In [175]:
Venue = []
City = []
def venue_city(venue_links):
    '''
    This Function will Extract the Venue & City Names from Venu_links
    Input : Venue HTML tag
    Output : Venue & City in Separate list(s)


    Sample Input : 'Narendra Modi Stadium, Ahmedabad'
    '''

    # For Easy Understanding please see above Input
    for i in range(1,len(venue_links),2): # please see above note
        venue_city = venue_links[i].text  # Venue_City Extraction
        venue = venue_city.split(',')[0].strip() # Venue Extraction
        city = venue_city.split(',')[1].strip()  # City Extraction

        Venue.append(venue)
        City.append(city)


    # Crosscheck
    print(f'Venue Details : {Venue[:3]} {len(Venue)}')
    print(f'City Details : {City[:3]} {len(City)}')

venue_city(venue_links)

Venue Details : ['MA Chidambaram Stadium', 'Maharaja Yadavindra Singh International Cricket Stadium', 'Eden Gardens'] 74
City Details : ['Chennai', 'Mullanpur', 'Kolkata'] 74


### Player of the Match

In [176]:
def player_of_the_match_extraction(url):
    '''
    This Function will return the POTM (Player of the Match) for Each Match
    Input : Each Match url
    Output : POTM Name

    Sample Input :
    '''


    potm_soup = requests.get(url)
    potm_soup_html = BeautifulSoup(potm_soup.content,'html.parser')
    potm_soup_html_obj = potm_soup_html.find_all('a',attrs={'class':'cb-link-undrln'}) #POTM Class
    player = BeautifulSoup(str(potm_soup_html_obj), 'html.parser')

    return player.a.get_text() if player.a else "Match abandoned"


## Match Winner

In [177]:
def match_winner(url):
  soup = BeautifulSoup(requests.get(url).content,'html.parser')
  winner = soup.find('div',attrs={'class':'cb-col cb-col-100 cb-min-stts cb-text-complete'})
  if bool(winner):
    w_index = winner.text.find('won')
    winner = winner.text[:w_index].strip()
    return "".join([word[0] for word in winner.split(' ')])
  else:
    return "Match abandoned"

### Match Info

In [178]:
Match_info = []
player_of_the_match = []
winner = []
Score_card = []

In [179]:
cricbuzz = 'https://www.cricbuzz.com/live-cricket-full-commentary/'
potm_url = 'https://www.cricbuzz.com/cricket-scores/'
score_card = 'https://www.cricbuzz.com/live-cricket-scorecard/'
cnt = 1
for match_ in match_links[:74]:
    semi_url_index = match_.get('href').find('/',3)
    url = match_.get('href')[semi_url_index:]
    match_info = cricbuzz + url
    potm = potm_url + url
    scorecard = score_card+url
    
    Match_info.append(match_info)
    player_of_the_match.append(player_of_the_match_extraction(potm))
    Score_card.append(scorecard)
  #winner.append(match_winner(potm))

### Cross Checking

In [180]:
len(match_List),len(team_1_List),len(team_2_List),len(Match_info),len(player_of_the_match),len(Venue),len(City)

(74, 74, 74, 74, 74, 74, 74)

In [181]:
Match_info[:3]

['https://www.cricbuzz.com/live-cricket-full-commentary//89654/csk-vs-rcb-1st-match-indian-premier-league-2024',
 'https://www.cricbuzz.com/live-cricket-full-commentary//89661/pbks-vs-dc-2nd-match-indian-premier-league-2024',
 'https://www.cricbuzz.com/live-cricket-full-commentary//89665/kkr-vs-srh-3rd-match-indian-premier-league-2024']

In [182]:
Score_card[-3:]

['https://www.cricbuzz.com/live-cricket-scorecard//91731/rr-vs-rcb-eliminator-indian-premier-league-2024',
 'https://www.cricbuzz.com/live-cricket-scorecard//91735/srh-vs-rr-qualifier-2-indian-premier-league-2024',
 'https://www.cricbuzz.com/live-cricket-scorecard//91740/kkr-vs-srh-final-indian-premier-league-2024']

In [183]:
len(match_List)

74

### dataframe creation

In [184]:
df = pd.DataFrame({'Match':match_List,'Team_1':team_1_List,'Team_2':team_2_List,'Player_of_the_Match':player_of_the_match,'Venue':Venue,'City':City,'Match_info':Match_info,'Score_Card':Score_card})
df.head()

,Match,Team_1,Team_2,Player_of_the_Match,Venue,City,Match_info,Score_Card
0,CHENNAI SUPER KINGS VS ROYAL CHALLENGERS BENGA...,CSK,RCB,Mustafizur Rahman,MA Chidambaram Stadium,Chennai,https://www.cricbuzz.com/live-cricket-full-com...,https://www.cricbuzz.com/live-cricket-scorecar...
1,PUNJAB KINGS VS DELHI CAPITALS,PK,DC,Sam Curran,Maharaja Yadavindra Singh International Cricke...,Mullanpur,https://www.cricbuzz.com/live-cricket-full-com...,https://www.cricbuzz.com/live-cricket-scorecar...
2,KOLKATA KNIGHT RIDERS VS SUNRISERS HYDERABAD,KKR,SH,Andre Russell,Eden Gardens,Kolkata,https://www.cricbuzz.com/live-cricket-full-com...,https://www.cricbuzz.com/live-cricket-scorecar...
3,RAJASTHAN ROYALS VS LUCKNOW SUPER GIANTS,RR,LSG,Sanju Samson,Sawai Mansingh Stadium,Jaipur,https://www.cricbuzz.com/live-cricket-full-com...,https://www.cricbuzz.com/live-cricket-scorecar...
4,GUJARAT TITANS VS MUMBAI INDIANS,GT,MI,Sai Sudharsan,Narendra Modi Stadium,Ahmedabad,https://www.cricbuzz.com/live-cricket-full-com...,https://www.cricbuzz.com/live-cricket-scorecar...


In [185]:
df['Team_1'].unique(),df['Team_2'].unique()

(array(['CSK', 'PK', 'KKR', 'RR', 'GT', 'RCB', 'SH', 'LSG', 'DC', 'MI'],
       dtype=object),
 array(['RCB', 'DC', 'SH', 'LSG', 'MI', 'PK', 'GT', 'KKR', 'CSK', 'RR'],
       dtype=object))

In [186]:
df['Team_1'].rename({'PK':'PBKS','SH':'SRH'},inplace=True)
df['Team_2'].rename({'PK':'PBKS','SH':'SRH'},inplace=True)

In [187]:
df['Team_1'].unique(),df['Team_2'].unique()

(array(['CSK', 'PK', 'KKR', 'RR', 'GT', 'RCB', 'SH', 'LSG', 'DC', 'MI'],
       dtype=object),
 array(['RCB', 'DC', 'SH', 'LSG', 'MI', 'PK', 'GT', 'KKR', 'CSK', 'RR'],
       dtype=object))

In [155]:
df.to_csv('schedule.csv',index=False)